# This file is for developing and using yolov5 for detecting images on test machines

## Setup

Check setup for use.

In [ ]:
from yolov5 import utils
display = utils.notebook_init()  # checks

## Detect chess pieces on picture and return prediction

This document uses`detect.py` and saves `runs/detect`. Example inference sources are:

```shell
python detect.py --source 0  # webcam
                          img.jpg  # image
                          vid.mp4  # video
                          path/  # directory
                          path/*.jpg  # glob
                          'https://youtu.be/Zgi9g1ksQHc'  # YouTube
                          'rtsp://example.com/media.mp4'  # RTSP, RTMP, HTTP stream
```

The following command will try to detect chess pieces on a test picture not known to the model in `../testfiles/<img.jpg>`.

All the trained models can be found either in their run in `yolov5/runs/train/<run>/weights/<best.pt>` or if they are used in detection also in `weights/customWeights/<model.pt>`.
They are named after the convention: `<modelName>_<run>`. An example would be: `best_300epochsYolov5x`.
```shell
!python3 detect.py --weights <path>/<chessModel.pt> # the model that is used to detect the objects on the image
                   --img 416 # the dimensions of the image
                   --conf 0.25 # the confidence threshhold that is set
                   --source ../testfiles/<img.jpg> # the path the the image that is used for detection
```

In [ ]:
# Run inference on test dataset; best so far: best_Big_300ep_yolov5x-X

#!python3 detect.py --weights weights/customWeights/best_Big_150ep_yolov5n.pt --img 416 --conf 0.25 --source ../datasets/Chess-Pieces_big/test/images/
#!python3 detect.py --weights weights/customWeights/last_Big_150ep_yolov5n.pt --img 416 --conf 0.25 --source ../datasets/Chess-Pieces_big/test/images/
#!python3 detect.py --weights weights/customWeights/best_Big_300ep_yolov5n.pt --img 416 --conf 0.25 --source ../datasets/Chess-Pieces_big/test/images/
#!python3 detect.py --weights weights/customWeights/last_Big_300ep_yolov5n.pt --img 416 --conf 0.25 --source ../datasets/Chess-Pieces_big/test/images/
#!python3 detect.py --weights weights/customWeights/best_Big_150ep_yolov5x.pt --img 416 --conf 0.25 --source ../datasets/Chess-Pieces_big/test/images/
#!python3 detect.py --weights weights/customWeights/last_Big_150ep_yolov5x.pt --img 416 --conf 0.25 --source ../datasets/Chess-Pieces_big/test/images/
!python3 detect.py --weights weights/customWeights/best_Big_300ep_yolov5x.pt --img 416 --conf 0.25 --source ../datasets/Chess-Pieces_big/test/images/
!python3 detect.py --weights weights/customWeights/last_Big_300ep_yolov5x.pt --img 416 --conf 0.25 --source ../datasets/Chess-Pieces_big/test/images/

Show the image with the detected objects.

In [ ]:
from IPython.display import Image
display.Image(filename='runs/detect/exp/zidane.jpg', width=416)

## Preprocess test files before processing with model

In [3]:
import glob
from PIL import Image, ImageOps
count = 1
images = []
for f in glob.glob("../testfiles/*.jpeg"):
    im = Image.open(f)
    images.append(im)

for f in images:
    f = ImageOps.exif_transpose(f)
    f.resize((416, 416))
    f.save('../testFiles/stretched/IMG'+str(count)+'.jpeg')
    print(f.size) # Output: (416, 416)
    count+=1

(3024, 4032)
(3024, 4032)
(3024, 4032)


In [ ]:
#Best so far
!python3 detect.py --weights weights/customWeights/best_Big_300ep_yolov5x.pt --img 416 --conf 0.25 --source ../testFiles/stretched

# Pyhton method to use detection

```shell
model.conf = 0.25  # NMS confidence threshold
      iou = 0.45  # NMS IoU threshold
      agnostic = False  # NMS class-agnostic
      multi_label = False  # NMS multiple labels per box
      classes = None  # (optional list) filter by class, i.e. = [0, 15, 16] for COCO persons, cats and dogs
      max_det = 1000  # maximum number of detections per image
      amp = False  # Automatic Mixed Precision (AMP) inference
```

In [ ]:
import torch, cv2
from PIL import Image

#Model
model = torch.hub.load('ultralytics/yolov5', 'custom', path='weights/customWeights/best_Big_300ep_yolov5x.pt')  # local model
model.conf = 0.25

In [ ]:
# Images
img1 = Image.open('../testFiles/stretched/IMG1.jpeg')  # PIL image
img2 = Image.open('../testFiles/stretched/IMG2.jpeg')  # PIL image
img3 = cv2.imread('../testFiles/stretched/IMG3.jpeg')[..., ::-1]  # OpenCV image (BGR to RGB)
imgs = [img1, img2, img3]  # batch of images


In [ ]:
# Inference
results = model(imgs, size=416)  # includes NMS

In [ ]:
# Results
results.print()
#results.save()  # or .show()

print(results.xyxy[0])  # img1 predictions (tensor)


In [ ]:
print(results.pandas().xyxy[0])  # img1 predictions (pandas)

In [ ]:
results.pandas().xyxy[0].to_json(orient="records")  # JSON img1 predictions